In [1]:
import pandas as pd
from Bio import SeqIO, AlignIO, Seq
import numpy as np
from collections import Counter
from helper import *
import python_cipres.client as CipresClient

### Cluster sequences using uclust

In [3]:
! mkdir -p ../output_itai/01_70p_tree
!../bin/usearch11.0.667_i86linux32 -cluster_fast ../output_itai/00_100p_tree/uclust_all_1.faa -id 0.7 -uc ../output_itai/01_70p_tree/uclust_all_0.7.uc

usearch v11.0.667_i86linux32, 4.0Gb RAM (16.3Gb total), 8 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: yinonmoise.baron@weizmann.ac.il

00:00 71Mb    100.0% Reading ../output_itai/00_100p_tree/uclust_all_1.faa
00:00 110Mb   100.0% DF
00:00 111Mb  50020 seqs, 50020 uniques, 50020 singletons (100.0%)
00:00 111Mb  Min size 1, median 1, max 1, avg 1.00
00:00 117Mb   100.0% DB
00:03 181Mb   100.0% 817 clusters, max size 20339, avg 61.2
                                                           
      Seqs  50020 (50.0k)
  Clusters  817
  Max size  20339 (20.3k)
  Avg size  61.2
  Min size  1
Singletons  409, 0.8% of seqs, 50.1% of clusters
   Max mem  181Mb
      Time  3.00s
Throughput  16.7k seqs/sec.



### Take only cluster fasta files and create csv file

In [4]:
parse_uclust(infile='../output_itai/01_70p_tree/uclust_all_0.7.uc',
             fasta='../output_itai/00_100p_tree/uclust_all_1.faa',
             outfasta='../output_itai/01_70p_tree/uclust_all_0.7.faa',
             outfile='../output_itai/01_70p_tree/uclust_all_0.7.csv'
            )

## Create multiple sequence alignment

In [5]:
!../bin/mafft-linux64/mafft.bat ../output_itai/01_70p_tree/uclust_all_0.7.faa > ../output_itai/01_70p_tree/uclust_all_0.7.aln

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
  801 / 817
done.

Constructing a UPGMA tree (efffree=0) ... 
  810 / 817
done.

Progressive alignment 1/2... 
STEP   501 / 816 
Reallocating..done. *alloclen = 2429
STEP   701 / 816  h
Reallocating..done. *alloclen = 3501
STEP   801 / 816  h
done.

Making a distance matrix from msa.. 
  800 / 817
done.

Constructing a UPGMA tree (efffree=1) ... 
  810 / 817
done.

Progressive alignment 2/2... 
STEP   501 / 816 
Reallocating..done. *alloclen = 2433
STEP   701 / 816  h
Reallocating..done. *alloclen = 3539
STEP   801 / 816  h
done.

disttbfast (aa) Version 7.427
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'maf

## Clean MSA to contain only positions with more than 5% coverage (based on Jaffe et al. 2018)

In [6]:
clean_aln(infile='../output_itai/01_70p_tree/uclust_all_0.7.aln',
          outfile='../output_itai/01_70p_tree/uclust_all_0.7_trimmed.aln')

### Run RaxML on CIPRES

In [7]:
cip = CipresClient.Client(appname='RO',
                    appID='rubisco_phylogeny-49F87B124F3D429FBE12F95E4254DDEA',
                    baseUrl='https://cipresrest.sdsc.edu/cipresrest/v1',
                    username='yinonbaron',
                    password='Mchcav11~')


In [8]:

job = cip.submitJob(vParams={'toolId': 'RAXMLHPC8_REST_XSEDE',
                       'datatype_': 'protein',
                       'runtime_': '160',
                       'select_analysis_': 'fa',
                       'choose_bootstrap_': 'x',
                       'printbrlength_': '1'},
              inputParams={'infile_': '../output_itai/01_70p_tree/uclust_all_0.7_trimmed.aln'},
              metadata={'statusEmail':'true'})

In [11]:
if cip.getJobStatus(jobHandle=job.jobHandle).isDone():
    !mkdir -p ../output_itai/01_70p_tree/RaxML/
    job.downloadResults('../output_itai/01_70p_tree/RaxML/')
else:
    print('Job ' + job.jobHandle + ' not finished')

In [3]:
#!curl -u yinonbaron:Mchcav11~ -H cipres-appkey:rubisco_phylogeny-49F87B124F3D429FBE12F95E4254DDEA https://cipresrest.sdsc.edu/cipresrest/v1/job/yinonbaron -F tool=RAXMLHPC8_REST_XSEDE  -F input.infile_=@../output/01_70p_tree/uclust_all_0.7_trimmed.aln -F metadata.statusEmail=true -F vparam.datatype_=protein -F vparam.runtime_=160 -F  vparam.select_analysis_=fa -F vparam.choose_bootstrap_=x -F vparam.printbrlength_=1

In [3]:
# Optional - create tree using fasttree
#!../bin/FastTree -spr 4 -mlacc 2 -slownni ../output/03_uclust_all_0.7_trimmed.aln > ../output/04_uclust_all_0.7.nwk

In [12]:
add_type(type_file='../output_itai/00_100p_tree/uclust_all_1_rubisco_types.csv',
         seq_file='../output_itai/01_70p_tree/uclust_all_0.7.csv',
         outfile='../output_itai/01_70p_tree/type_legend.txt')

In [22]:
kinetic_data = pd.read_csv('../output_itai/00_100p_tree/uclust_all_1_kinetic_data.csv',names=['kinetic_ID'])
uc = pd.read_csv('../output_itai/01_70p_tree/uclust_all_0.7.csv')
uc2 = uc.merge(kinetic_data,left_on='Query', right_on='kinetic_ID')
uc2['kinetic_flag'] = '-1'
uc2.loc[~uc2.kinetic_ID.isna(),'kinetic_flag'] = '1'
lines = uc2['Target'].apply(lambda x: x.split(' ')[0]).values + ','+ uc2['kinetic_flag'].values+',-1\n'
#unique_lines = np.unique(lines)

In [23]:

with open('../data/kinetic_sampling_legend.txt','r') as file:
    with open('../output_itai/01_70p_tree/kinetic_sample.txt', "w") as f1:
        for row in file:
            f1.write(row)
        for line in lines:
            f1.write(line)
        file.close()
        f1.close()

In [29]:
type_file='../output_iti/00_100p_tree/uclust_all_1_rubisco_types.csv'
seq_file='../output_itai/01_70p_tree/uclust_all_0.7.csv'
outfile='../output_itai/01_70p_tree/type_legend.txt'

labels = pd.read_csv(type_file)
uclust_data = pd.read_csv(seq_file)
uclust_data = uclust_data[uclust_data['Type'] !='S']
uclust_data.loc[uclust_data['Target'] == '*','Target'] = uclust_data.loc[uclust_data['Target'] == '*','Query']
uclust_data = uclust_data.merge(labels, left_on='Query', right_on='ID',how='left')
labeled_leaves = uclust_data[~pd.isna(uclust_data['type'])]
color_map = {'I': '#28B463',
             'II': '#E74C3C',
             'II/III':'#AF7AC5',
             'IIIa':'#AED6F1',
             'IIIb':'#3498DB',
             'IIIc':'#1F618D',
             'IIIlike':'#5D6D7E',
             'IV':'#F4D03F',
             'IVlike':'#F8C471',
             'unknown':'#F442D4'}
lines = labeled_leaves['Target'].apply(lambda x: x.split(' ')[0]).values +[',label,node,'+color_map[x]+',1,normal\n' for x in labeled_leaves['type'].values]
lines = set(lines)
'''
with open('../data/itol_legend_template.txt','r') as file:
    with open(outfile, "w") as f1:
        for row in file:
            f1.write(row)
        for line in lines:
            f1.write(line)
        file.close()
        f1.close()
        '''

'\nwith open(\'../data/itol_legend_template.txt\',\'r\') as file:\n    with open(outfile, "w") as f1:\n        for row in file:\n            f1.write(row)\n        for line in lines:\n            f1.write(line)\n        file.close()\n        f1.close()\n        '